In [134]:
from skimage.feature import hog
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
import numpy as np
import cv2
import os
from PIL import Image
import matplotlib.pyplot as plt
from skimage import exposure
import sys
class Image:
    def __init__(self, img, label):
        self.img = img
        self.label = label

def Load_Dataset():
    currpath="Data/Product Classification"
    Classes=os.listdir(currpath)
    Classes.sort( key=lambda x: int(x))
    Classes=list(map(lambda x:currpath+"/"+x,Classes))
    X_train,y_train,X_valid,y_valid= [], [], [], []
    trainset=[]
    validset=[]
    mx=(0,0)
    for Class in Classes:
        trainpart=os.listdir(Class+"/"+"Train")
        trainpart=list(map(lambda x:Class+"/"+"Train"+"/"+x,trainpart))
        for file in trainpart:
            img=cv2.imread(file,0)
            label=Class.split("/")[-1]
            mx=max(mx,img.shape)
            trainset.append(Image(img,label))
            X_train.append(img)
            y_train.append(label)
        if "Validation"  not in os.listdir(Class):continue
        validpart=os.listdir(Class+"/"+"Validation")
        validpart=list(map(lambda x:Class+"/"+"Validation"+"/"+x,validpart))
        for file in validpart:
            img=cv2.imread(file,0)
            label=Class.split("/")[-1]
            validset.append(Image(img,label))
            X_valid.append(img)
            y_valid.append(label)
    return trainset,validset,X_train,y_train,X_valid,y_valid


def preprocessing(DataSet):
    processed_images = []
    labels = []

    for image in DataSet:
        img = image.img
        # Resize the image
        # Normalize the image
        normalized_image = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
        # Gaussian Blur
        blurred_image = normalized_image
        blurred_image=cv2.resize(blurred_image,(128,128))
        processed_images.append(blurred_image)
        labels.append(image.label)

    return processed_images, labels

    
def read_files_in_folder(folder_path):
    try:
        X_test = []
        y_test = []
        testset= []
        for label, class_name in enumerate(os.listdir(folder_path)):
            class_path = os.path.join(folder_path, class_name)
            for file_name in os.listdir(class_path):
                file_path = os.path.join(class_path, file_name)
                if os.path.isfile(file_path):
                    image = cv2.imread(file_path,0)
                    X_test.append(image)
                    y_test.append(class_name)
                    testset.append(Image(image,label))
        print("Hog Started")
        
        #    Step 3: HOG Feature Extraction
        X_test,y_test = preprocessing(testset)
        print("imgShape")
        print(X_test[0].shape)
        for i,image in enumerate(X_test) :
        # Calculate HOG features for each image
            if i==0:
                 features, hog_image = hog(image, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2, 2), visualize=True, block_norm='L2-Hys')
                 Test_hog_features =np.array(features) 
                 continue
            features, hog_image = hog(image, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2, 2), visualize=True, block_norm='L2-Hys')
            Test_hog_features=np.vstack((Test_hog_features,features))
        
        print("Hog Ended")

        return testset,Test_hog_features,y_test    
    except OSError as e:
        print(f"Error reading files in {folder_path}: {e}")
    


trainset,validset,X_train,y_train,X_valid,y_valid=Load_Dataset()
X_train_processed, y_train_processed = preprocessing(trainset)

X_valid_processed, y_valid_processed = preprocessing(validset)

In [135]:
trainset,validset,X_train,y_train,X_valid,y_valid=Load_Dataset()
X_train_processed, y_train_processed = preprocessing(trainset)
X_valid_processed, y_valid_processed = preprocessing(validset)


In [136]:
len(y_valid_processed)


34

In [137]:
# Assuming you have a dataset 'images' and corresponding 'labels'
Train_images=X_train_processed
Train_labels=y_train_processed

Valid_images=X_valid_processed
Valid_labels=y_valid_processed

print("Hog Started")
# Step 3: HOG Feature Extraction
 
for i,image in enumerate(Train_images) :
    # Calculate HOG features for each image
    if i==0:
        features, hog_image = hog(image, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2, 2), visualize=True, block_norm='L2-Hys')
        Train_hog_features =np.array(features)
        continue
    features, hog_image = hog(image, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2, 2), visualize=True, block_norm='L2-Hys')
    Train_hog_features=np.vstack((Train_hog_features,features))
print("Hog Ended")


print("Hog Started")
Valid_hog_features = []
for i,image in enumerate(Valid_images):
    # Calculate HOG features for each image
    if i==0:
        features, hog_image = hog(image, orientations=9, pixels_per_cell=(8, 8),
                              cells_per_block=(2, 2), visualize=True, block_norm='L2-Hys')
        Valid_hog_features =np.array(features)
        continue
    features, hog_image = hog(image, orientations=9, pixels_per_cell=(8, 8),
                              cells_per_block=(2, 2), visualize=True, block_norm='L2-Hys')
    Valid_hog_features=np.vstack((Valid_hog_features,features))    
    # Calculate HOG features for each image
 
print("Hog Ended")
testset,X_test,y_test=read_files_in_folder("E:/4th year projects/cv/Test Samples Classification/Test Samples Classification")


Hog Started
Hog Ended
Hog Started
Hog Ended
Hog Started
imgShape
(128, 128)
Hog Ended


In [141]:
# Step 5: Train-test Split
X_train, X_Valid, y_train, y_valid = np.array(Train_hog_features),np.array(Valid_hog_features),np.array(Train_labels,dtype=int),np.array(Valid_labels,dtype=int)

print("SVM Training")
# Step 6: SVM Model Training
svm_model = svm.SVC(kernel='rbf',random_state=2002,C=100)
svm_model.fit(X_train, y_train)

print("SVM Trained")

# Step 7: Testing and Evaluation
y_pred = svm_model.predict(X_train)
accuracy = accuracy_score(y_train, y_pred)
print("Accuracy Train:", accuracy)

y_pred = svm_model.predict(X_Valid)
accuracy = accuracy_score(y_valid, y_pred)
print("Accuracy Train:", accuracy)

y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy Train:", accuracy)


SVM Training
SVM Trained
Accuracy Train: 1.0
Accuracy Train: 0.5882352941176471
Accuracy Train: 0.047619047619047616
